In [1]:
from __future__ import annotations

TARGET_MODEL = "mistralai/Mistral-7B-v0.1"

DEBUG = False

In [2]:
# %% Directory settings

# ====================================================
# Directory settings
# ====================================================
from pathlib import Path

OUTPUT_DIR = Path("./")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("../input/")

In [3]:
import pandas as pd
train_df = pd.read_csv(INPUT_DIR / "llm-detect-ai-generated-text/train_essays.csv", sep=',')
test_df = pd.read_csv(INPUT_DIR / "llm-detect-ai-generated-text/test_essays.csv", sep=',')
external_df = pd.read_csv(INPUT_DIR / "daigt-external-dataset/daigt_external_dataset.csv", sep=',')
train_prompts_df = pd.read_csv(INPUT_DIR / "llm-detect-ai-generated-text/train_prompts.csv", sep=',')

# show shape
print(f'train_df.shape: {train_df.shape}')
print(f'test_df.shape: {test_df.shape}')
print(f'external_df.shape: {external_df.shape}')
print(f'train_prompts_df.shape: {train_prompts_df.shape}')

train_df.shape: (1378, 4)
test_df.shape: (3, 3)
external_df.shape: (2421, 4)
train_prompts_df.shape: (2, 4)


In [4]:
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/


In [5]:
# rename column generated to label
train_df = train_df.rename(columns={'generated': 'label'})
test_df = test_df.rename(columns={'generated': 'label'})
external_df = external_df.rename(columns={'generated': 'label'})


In [6]:
train_df.label.value_counts()

label
0    1375
1       3
Name: count, dtype: int64

In [7]:
train_df.head(3)

,id,prompt_id,text,label
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0


In [8]:
test_df.head(3)

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [9]:
external_df.head(3)

,id,text,instructions,source_text
0,6060D28C05B6,Some schools in United States ofter classes fr...,\nTask: Write a persuasive essay on whether or...,\nWhen considering the pros and cons of attend...
1,60623DB5DE7A,"Four-day work week, a remarkable idea to conse...",\nTask: Research the advantages and disadvanta...,\nOne of the primary arguments for implementin...
2,607A39D981DE,Students and their families should consider an...,\nTask: \n\n1. Talk to your parents before tak...,\nBefore making any decisions about getting in...


In [10]:
external_df.head(3).values

array([['6060D28C05B6',
        "Some schools in United States ofter classes from home because is good option to students . Some school have decreased bullying and high and middle school because some students get bullied. Students don't want to go more at school and they want to get classes at home. If they get classes at home they don't learning. Some Schools offter distance learning as an option for students to attend classes from home by way of online or video conferencing. students get distract at home. Students get fall in environment learning. Students feel more pressure at home. But students can get relaxing at home.\n\nStudents can get distract at home because they have easy to use phones. They feel more bored at home because they need stay at home more time. Also students don't do anything at home because if they stay at home is esay to get more distract. If students sleep in class they want to sleep at home too. Many students work and they would tired and they don't want to l

In [11]:
train_prompts_df.head(3).values

array([[0, 'Car-free cities',
        'Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.',
        '# In German Suburb, Life Goes On Without Cars by Elisabeth Rosenthal\n\n1 VAUBAN, Germany—Residents of this upscale community are suburban pioneers, going where few soccer moms or commuting executives have ever gone before: they have given up their cars.\n\n2 Street parking, driveways and home garages are generally forbidden in this experimental new district on the outskirts of Freiburg, near the French and Swiss borders. Vauban’s st

In [12]:
external_df = external_df[["id", "source_text"]]
external_df.columns = ["id", "text"]
external_df['text'] = external_df['text'].str.replace('\n', '')
external_df["label"] = 1
train_df.drop(columns=["prompt_id"],inplace=True)
train_df = pd.concat([train_df, external_df])
train_df.reset_index(inplace=True, drop=True)
print(f"Train dataframe has shape: {train_df.shape}")
train_df.head()

Train dataframe has shape: (3799, 3)


,id,text,label
0,0059830c,Cars. Cars have been around since they became ...,0
1,005db917,Transportation is a large necessity in most co...,0
2,008f63e3,"""America's love affair with it's vehicles seem...",0
3,00940276,How often do you ride in a car? Do you drive a...,0
4,00c39458,Cars are a wonderful thing. They are perhaps o...,0


In [13]:
train_df.value_counts("label")

label
1    2424
0    1375
Name: count, dtype: int64

In [14]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train_df.loc[:, train_df.columns != "label"]
y = train_df.loc[:, train_df.columns == "label"]

for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    train_df.loc[valid_index, "fold"] = i
    
print(train_df.groupby("fold")["label"].value_counts())
train_df.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


fold  label
0.0   1        485
      0        275
1.0   1        485
      0        275
2.0   1        485
      0        275
3.0   1        485
      0        275
4.0   1        484
      0        275
Name: count, dtype: int64


,id,text,label,fold
0,0059830c,Cars. Cars have been around since they became ...,0,1.0
1,005db917,Transportation is a large necessity in most co...,0,0.0
2,008f63e3,"""America's love affair with it's vehicles seem...",0,3.0
3,00940276,How often do you ride in a car? Do you drive a...,0,2.0
4,00c39458,Cars are a wonderful thing. They are perhaps o...,0,0.0


In [15]:
# fold0 as valid
valid_df = train_df[train_df["fold"] == 0]
train_df = train_df[train_df["fold"] != 0]


In [16]:
# load model with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [17]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [18]:

base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=2,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id

You are using a model of type mistral to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model = get_peft_model(base_model, peft_config)


In [20]:
model.print_trainable_parameters()


trainable params: 27,279,360 || all params: 7,137,939,456 || trainable%: 0.38217415779661107


In [21]:
# debug
if DEBUG:
    train_df = train_df.sample(300)
    valid_df = valid_df.sample(50)
#train_df = train_df.sample(100)
#valid_df = valid_df.sample(30)
print(train_df.label.value_counts(), valid_df.label.value_counts())

label
1    1939
0    1100
Name: count, dtype: int64 label
1    485
0    275
Name: count, dtype: int64


In [22]:
# datasets
from datasets import Dataset

# from pandas
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)

In [23]:
def preprocess_function(examples, max_length=512):
    return tokenizer(examples["text"], truncation=True, max_length=max_length, padding=True)

In [24]:
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [26]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy_val = accuracy_score(labels, predictions)
    roc_auc_val = roc_auc_score(labels, predictions)
    
    return {
        "accuracy": accuracy_val,
        "roc_auc": roc_auc_val,
    }


In [27]:
from transformers import TrainingArguments, Trainer

steps = 5 if DEBUG else 20

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    max_grad_norm=0.3,
    optim='paged_adamw_32bit',
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=False,
    warmup_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    report_to='none' # if DEBUG else 'wandb',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_ds,
    eval_dataset=valid_tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Roc Auc
20,6.029700,1.883789,0.755263,0.682287
40,0.853600,0.171997,0.963158,0.952240
60,0.121400,0.027267,0.990789,0.990422
80,0.008900,0.006062,0.996053,0.995333
100,0.039900,0.001627,0.998684,0.998182
120,0.028500,0.001989,0.998684,0.998182
140,0.000000,0.002234,0.998684,0.998182
160,0.015500,0.002302,0.998684,0.998182
180,0.000000,0.002310,0.998684,0.998182


TrainOutput(global_step=189, training_loss=0.7510537719600415, metrics={'train_runtime': 21444.8687, 'train_samples_per_second': 0.142, 'train_steps_per_second': 0.009, 'total_flos': 6.499351637920973e+16, 'train_loss': 0.7510537719600415, 'epoch': 1.0})

In [28]:
from shutil import rmtree

trainer.save_model(output_dir=str(OUTPUT_DIR))

for path in Path(training_args.output_dir).glob("checkpoint-*"):
    if path.is_dir():
        rmtree(path)

In [29]:
del trainer, model, base_model

In [30]:
# cuda cache clear
import torch
torch.cuda.empty_cache()

In [31]:
# load model / tokenizer with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [32]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=2,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id


You are using a model of type mistral to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
model = PeftModel.from_pretrained(base_model, str(OUTPUT_DIR))


In [34]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [35]:
pred_output = trainer.predict(valid_tokenized_ds)
logits = pred_output.predictions

In [36]:
logits = pred_output.predictions

In [37]:
valid_df.label

1       0
4       0
8       0
9       0
15      0
       ..
3780    1
3782    1
3789    1
3796    1
3798    1
Name: label, Length: 760, dtype: int64

In [38]:
logits

array([[ 16.94 ,  -6.875],
       [  1.819, -17.8  ],
       [ 10.04 , -19.52 ],
       ...,
       [-21.67 ,  14.53 ],
       [-21.4  ,  20.84 ],
       [-31.45 ,  12.57 ]], dtype=float16)

In [39]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
probs = sigmoid(logits[:, 1])
probs.shape, probs[0:5]

/tmp/ipykernel_26/1101527902.py:4: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


((760,),
 array([0.001032, 0.      , 0.      , 0.      , 0.01234 ], dtype=float16))

In [40]:
sub = pd.DataFrame()
sub['id'] = valid_df['id']
sub['generated'] = probs
# sub.to_csv('submission.csv', index=False)
sub.head()

,id,generated
1,005db917,0.001032
4,00c39458,0.000000
8,01c6e176,0.000000
9,0202ddf9,0.000000
15,03ed46ca,0.012337


In [41]:
!ls -alh /kaggle/working/

total 105M
drwxr-xr-x 3 root root 4.0K Nov  1 18:55 .
drwxr-xr-x 6 root root 4.0K Nov  1 12:50 ..
-rw-r--r-- 1 root root  464 Nov  1 18:54 README.md
---------- 1 root root 100K Nov  1 19:11 __notebook__.ipynb
-rw-r--r-- 1 root root  445 Nov  1 18:54 adapter_config.json
-rw-r--r-- 1 root root 105M Nov  1 18:54 adapter_model.bin
-rw-r--r-- 1 root root  437 Nov  1 18:54 special_tokens_map.json
drwxr-xr-x 2 root root 4.0K Nov  1 18:55 tmp_trainer
-rw-r--r-- 1 root root 482K Nov  1 18:54 tokenizer.model
-rw-r--r-- 1 root root  968 Nov  1 18:54 tokenizer_config.json
-rw-r--r-- 1 root root 4.0K Nov  1 18:54 training_args.bin
